## Final Assignment

### Introduction
Complete Genome of a Family of five - Two Parents, Three Siblings (Genome Phenotype SNPs Raw Data)

Genomics is a branch of molecular biology that involves structure, function, variation, evolution and mapping of genomes. There are several companies offering next generation sequencing of human genomes from complete 3 billion base-pairs to a few thousand Phenotype SNPs. I have used 23andMe (using Illumina HumanOmniExpress-24) for this family's DNA’s Phenotype SNPs.

### The question :
I going to find How similar are the SNPs genomes of parents and their children?

### About Data:
This dataset selected from the kaggle website. (link mentioned)

https://www.kaggle.com/datasets/zusmani/family-genome-dataset

The dataset is a complete genome of a Family of five - Two Parents, Three Siblings (Genome Phenotype SNPs Raw Data)which have been analysed only two children with their Father.The dataset is represented as a sequence of SNPs represented by the following symbols: A (adenine), C (cytosine), G (guanine), T (thymine). It contains Chromosomes 1-22, X, Y, and mitochondrial DNA.

### Import needed Library

In [1]:
import pandas as pd
import numpy as np
import yaml
from bokeh.io import output_notebook
from bokeh.plotting import ColumnDataSource
from bokeh.palettes import Bright6
from bokeh.plotting import figure, show, output_notebook
from bokeh.palettes import HighContrast3
from functools import reduce
from bokeh.palettes import Spectral
from bokeh.palettes import Greys256, Inferno256, Magma256,Plasma256
from bokeh.palettes import Viridis256, Cividis256, Turbo256
from bokeh.transform import dodge
from bokeh.models import TabPanel, Tabs

output_notebook()

Loading BokehJS ...

### Load and select Data

In [2]:
# load config file
def get_config():
    with open("Final_Assignment2.yaml", 'r') as stream:
        config = yaml.safe_load(stream)
    return config
config = get_config()

#load and read data frames
Child1Genome = config["Child1Genome"]
Child2Genome = config["Child2Genome"]
Child3Genome = config["Child3Genome"]
FatherGenome = config["FatherGenome"]
MotherGenome = config["MotherGenome"]

#useing low_memory=False because the Data for each column is very memory demanding.
df_Child1Genome = pd.read_csv(Child1Genome, low_memory=False)
df_Child2Genome = pd.read_csv(Child2Genome, low_memory=False)
df_Child3Genome = pd.read_csv(Child3Genome, low_memory=False)
df_FatherGenome = pd.read_csv(FatherGenome, low_memory=False)
df_MotherGenome = pd.read_csv(MotherGenome, low_memory=False)

#### lets take a look at the data

In [3]:
# Monitoring Data
print(df_Child1Genome.head())
print(df_Child2Genome.head())
print(df_FatherGenome.head())
df_Child1Genome.describe()
df_Child1Genome.info()

        # rsid chromosome  position genotype
0   rs12564807          1    734462       AA
1    rs3131972          1    752721       AG
2  rs148828841          1    760998       AC
3   rs12124819          1    776546       AG
4  rs115093905          1    787173       GG
        # rsid chromosome  position genotype
0  rs548049170          1     69869       TT
1    rs9283150          1    565508       AA
2  rs116587930          1    727841       GG
3    rs3131972          1    752721       GG
4   rs12184325          1    754105       CC
        # rsid chromosome  position genotype
0   rs12564807          1    734462       AA
1    rs3131972          1    752721       AG
2  rs148828841          1    760998       AC
3   rs12124819          1    776546       AA
4  rs115093905          1    787173       GG
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601802 entries, 0 to 601801
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   -----

### Data Preproccessing

#### Noise Detection
As we know genotype should be charectors.

In [4]:
# finding noise value
print(df_Child1Genome[~df_Child1Genome['genotype'].str.isalpha()].genotype)
print(df_Child2Genome[~df_Child2Genome['genotype'].str.isalpha()].genotype)
print(df_Child3Genome[~df_Child3Genome['genotype'].str.isalpha()].genotype)
print(df_FatherGenome[~df_FatherGenome['genotype'].str.isalpha()].genotype)
print(df_MotherGenome[~df_MotherGenome['genotype'].str.isalpha()].genotype)


51        --
113       --
114       --
117       --
128       --
          ..
601748    --
601749    --
601772    --
601776    --
601777    --
Name: genotype, Length: 9224, dtype: object
115       --
125       --
194       --
238       --
266       --
          ..
631905    --
631906    --
631907    --
631965    --
631966    --
Name: genotype, Length: 14590, dtype: object
64        --
115       --
125       --
151       --
194       --
          ..
631889    --
631890    --
631905    --
631906    --
631907    --
Name: genotype, Length: 16025, dtype: object
51        --
128       --
158       --
166       --
171       --
          ..
601747    --
601748    --
601749    --
601772    --
601773    --
Name: genotype, Length: 6616, dtype: object
32        --
51        --
53        --
110       --
111       --
          ..
601747    --
601748    --
601749    --
601772    --
601773    --
Name: genotype, Length: 11643, dtype: object


#### Noise Management
I replace the noise value to null. So I manage miss value later

In [5]:
# replace noise value to null

df_Child1Genome = df_Child1Genome.replace('--', np.nan)
df_Child2Genome = df_Child2Genome.replace('--', np.nan)
df_Child3Genome = df_Child3Genome.replace('--', np.nan)
df_FatherGenome = df_FatherGenome.replace('--', np.nan)
df_MotherGenome = df_MotherGenome.replace('--', np.nan)


#### Miss value detection
Finding miss values

In [6]:
# percentage of each dataset miss values

print(len(df_Child1Genome[df_Child1Genome['genotype'].isna()])/len(df_Child1Genome['genotype'])* 100)
print(len(df_Child2Genome[df_Child2Genome['genotype'].isna()])/len(df_Child2Genome['genotype'])* 100)
print(len(df_Child3Genome[df_Child3Genome['genotype'].isna()])/len(df_Child3Genome['genotype'])* 100)
print(len(df_FatherGenome[df_FatherGenome['genotype'].isna()])/len(df_FatherGenome['genotype'])* 100)
print(len(df_MotherGenome[df_MotherGenome['genotype'].isna()])/len(df_Child3Genome['genotype'])* 100)

1.532730034130827
2.3086064023874058
2.5356694721218767
1.0993649073947909
1.842296390883932


#### Miss value Management
Becuase the proportion of missing Data are less than 3 Persent, these would be droped

In [7]:
# drop all miss values
df_Child1Genome = df_Child1Genome.dropna()
df_Child2Genome = df_Child2Genome.dropna()
df_Child3Genome = df_Child3Genome.dropna()
df_FatherGenome = df_FatherGenome.dropna()
df_MotherGenome = df_MotherGenome.dropna()

### Data Analysis
To analysis similarity i should do flow road map.
- Data prepration
- Calculation similarity of each row
- aggrigation and visualization

#### Data prepration
- rearrenge columns
- rename columns
- merge dafa frames

In [8]:
#rearrenge the order of columns:
df_Child1Genome = df_Child1Genome[['chromosome', 'position', '# rsid', 'genotype']]
df_Child2Genome = df_Child2Genome[['chromosome', 'position', '# rsid', 'genotype']]
df_Child3Genome = df_Child3Genome[['chromosome', 'position', '# rsid', 'genotype']]
df_FatherGenome = df_FatherGenome[['chromosome', 'position', '# rsid', 'genotype']]
df_MotherGenome = df_MotherGenome[['chromosome', 'position', '# rsid', 'genotype']]

#renamed columns:
df_Child1Genome = df_Child1Genome.rename(columns={"# rsid":"rsid_child1","genotype":"genotype_child1"})
df_Child2Genome = df_Child2Genome.rename(columns={"# rsid":"rsid_child2","genotype":"genotype_child2"})
df_Child3Genome = df_Child3Genome.rename(columns={"# rsid":"rsid_child3","genotype":"genotype_child3"})
df_FatherGenome = df_FatherGenome.rename(columns={"# rsid":"rsid_father","genotype":"genotype_father"})
df_MotherGenome = df_MotherGenome.rename(columns={"# rsid":"rsid_mother","genotype":"genotype_mother"})

#print(df_Child1Genome.info())
#print(df_Child2Genome.info())
#print(df_Child3Genome.info())
#print(df_FatherGenome.info())
#print(df_MotherGenome.info())

# merge child to parent
Father_children_Genome = pd.merge(df_FatherGenome,df_Child1Genome,on=["chromosome","position"])
Father_children_Genome = pd.merge(Father_children_Genome,df_Child2Genome,on=["chromosome","position"])
Father_children_Genome = pd.merge(Father_children_Genome,df_Child3Genome,on=["chromosome","position"]) 

# merge child to parent
Mother_children_Genome = pd.merge(df_MotherGenome,df_Child1Genome,on=["chromosome","position"])
Mother_children_Genome = pd.merge(Mother_children_Genome,df_Child2Genome,on=["chromosome","position"])
Mother_children_Genome = pd.merge(Mother_children_Genome,df_Child3Genome,on=["chromosome","position"])

In [9]:
Father_children_Genome.head(5)

,chromosome,position,rsid_father,genotype_father,rsid_child1,genotype_child1,rsid_child2,genotype_child2,rsid_child3,genotype_child3
0,1,752721,rs3131972,AG,rs3131972,AG,rs3131972,GG,rs3131972,GG
1,1,776546,rs12124819,AA,rs12124819,AG,rs12124819,AA,rs12124819,AG
2,1,824398,rs7538305,AA,rs7538305,AA,rs7538305,AA,rs7538305,AA
3,1,846808,rs4475691,TT,rs4475691,CT,rs4475691,TT,rs4475691,CT
4,1,854250,rs7537756,GG,rs7537756,AG,rs7537756,GG,rs7537756,AG


In [10]:
Mother_children_Genome.head(5)

,chromosome,position,rsid_mother,genotype_mother,rsid_child1,genotype_child1,rsid_child2,genotype_child2,rsid_child3,genotype_child3
0,1,752721,rs3131972,GG,rs3131972,AG,rs3131972,GG,rs3131972,GG
1,1,776546,rs12124819,AG,rs12124819,AG,rs12124819,AA,rs12124819,AG
2,1,824398,rs7538305,AA,rs7538305,AA,rs7538305,AA,rs7538305,AA
3,1,846808,rs4475691,CT,rs4475691,CT,rs4475691,TT,rs4475691,CT
4,1,854250,rs7537756,AG,rs7537756,AG,rs7537756,GG,rs7537756,AG


#### Calculation similarity of each row

In [11]:
#similarity analysis by using Definition on Datasets
# ============== Father similarity ===============
def Father_similarity_child1(df):
    if df["genotype_father"] == df["genotype_child1"]:
        similarity_child1 = 1
    else:
        similarity_child1 = 0
    return similarity_child1 
       
def Father_similarity_child2(df):
    if df["genotype_father"] == df["genotype_child2"]:
        similarity_child2 = 1
    else:
        similarity_child2 = 0
    return similarity_child2 

def Father_similarity_child3(df):
    if df["genotype_father"] == df["genotype_child3"]:
        similarity_child3 = 1
    else:
        similarity_child3 = 0
    return similarity_child3 

Father_children_Genome["Father_similarity_child1"] = Father_children_Genome.apply(Father_similarity_child1,axis=1) 
Father_children_Genome["Father_similarity_child2"] = Father_children_Genome.apply(Father_similarity_child2,axis=1)  
Father_children_Genome["Father_similarity_child3"] = Father_children_Genome.apply(Father_similarity_child3,axis=1)   

# ============== Mother similarity ===============
def Mother_similarity_child1(df):
    if df["genotype_mother"] == df["genotype_child1"]:
        similarity_child1 = 1
    else:
        similarity_child1 = 0
    return similarity_child1 
       
def Mother_similarity_child2(df):
    if df["genotype_mother"] == df["genotype_child2"]:
        similarity_child2 = 1
    else:
        similarity_child2 = 0
    return similarity_child2 

def Mother_similarity_child3(df):
    if df["genotype_mother"] == df["genotype_child3"]:
        similarity_child3 = 1
    else:
        similarity_child3 = 0
    return similarity_child3 

Mother_children_Genome["Mother_similarity_child1"] = Mother_children_Genome.apply(Mother_similarity_child1,axis=1) 
Mother_children_Genome["Mother_similarity_child2"] = Mother_children_Genome.apply(Mother_similarity_child2,axis=1)  
Mother_children_Genome["Mother_similarity_child3"] = Mother_children_Genome.apply(Mother_similarity_child3,axis=1)

#### Visualizing Data
for the finding result, use bar chart with 3 value for 3 children

In [12]:
# aggrigate data

Father_children_Genome = Father_children_Genome.groupby("chromosome",as_index=False).aggregate({'Father_similarity_child1':'sum','Father_similarity_child2':'sum','Father_similarity_child3':'sum'})
Mother_children_Genome = Mother_children_Genome.groupby("chromosome",as_index=False).aggregate({'Mother_similarity_child1':'sum','Mother_similarity_child2':'sum','Mother_similarity_child3':'sum'})

In [13]:
legends = ["Child1","Child2","Child3"]

data1 = {'Similarity' : Father_children_Genome['chromosome'],
        'Child1' : Father_children_Genome["Father_similarity_child1"],
        'Child2' : Father_children_Genome["Father_similarity_child2"],
        'Child3' : Father_children_Genome["Father_similarity_child3"],}

source1 = ColumnDataSource(data=data1)

p1 = figure(x_range=Father_children_Genome['chromosome'], title="Similarity between children and father",
           height=350, toolbar_location=None, tools="")

p1.vbar(x=dodge('Similarity', -0.25, range=p1.x_range), top='Child1', source=source1,
       width=0.2, color="#e84d60", legend_label="Child1")

p1.vbar(x=dodge('Similarity',  0.0,  range=p1.x_range), top='Child2', source=source1,
       width=0.2, color="#718dbf", legend_label="Child2")

p1.vbar(x=dodge('Similarity',  0.25,  range=p1.x_range), top='Child3', source=source1,
       width=0.2, color="#71aabf", legend_label="Child3")

p1.x_range.range_padding = 0.1
p1.xgrid.grid_line_color = None
p1.legend.location = "top_left"
p1.legend.orientation = "horizontal"
p1.xaxis.major_label_orientation = "vertical"
p1.width = 1000
p1.height = 500


data2 = {'Similarity' : Mother_children_Genome['chromosome'],
        'Child1' : Mother_children_Genome["Mother_similarity_child1"],
        'Child2' : Mother_children_Genome["Mother_similarity_child2"],
        'Child3' : Mother_children_Genome["Mother_similarity_child3"],}

source2 = ColumnDataSource(data=data2)

p2 = figure(x_range=Father_children_Genome['chromosome'], title="Similarity between children and mother",
           height=350, toolbar_location=None, tools="")

p2.vbar(x=dodge('Similarity', -0.25, range=p2.x_range), top='Child1', source=source2,
       width=0.2, color="#e84d60", legend_label="Child1")

p2.vbar(x=dodge('Similarity',  0.0,  range=p2.x_range), top='Child2', source=source2,
       width=0.2, color="#718dbf", legend_label="Child2")

p2.vbar(x=dodge('Similarity',  0.25,  range=p2.x_range), top='Child3', source=source2,
       width=0.2, color="#71aabf", legend_label="Child3")

p2.x_range.range_padding = 0.1
p2.xgrid.grid_line_color = None
p2.legend.location = "top_left"
p2.legend.orientation = "horizontal"
p2.xaxis.major_label_orientation = "vertical"
p2.width = 1000
p2.height = 500

tab1 = TabPanel(child=p1, closable = True, title = "Father similarity")
tab2 = TabPanel(child=p2, closable = True, title = "Mother similarity")

show(Tabs(tabs = [tab1, tab2]))

#### Decision making
- As chart shows we find MT chromosome in child has most similarity for both between children-mother and children-father
- similarity between children and parents for 3 children is almost equal 


#### Next steps of question
I want to compare the similarity of the children with the father and the similarity of the children with the mother. So i need to calculate mean for each group.
Because 3 children silmilarity is the same so i going to calculate mean for father-children and mother-children

In [14]:
Father_child1_similarity_mean = (Father_children_Genome['Father_similarity_child1'].sum() + Father_children_Genome['Father_similarity_child1'].sum() + Father_children_Genome['Father_similarity_child1'].sum())/3
Mother_child1_similarity_mean = (Mother_children_Genome['Mother_similarity_child1'].sum() + Mother_children_Genome['Mother_similarity_child1'].sum() + Mother_children_Genome['Mother_similarity_child1'].sum())/3

print(Father_child1_similarity_mean,Mother_child1_similarity_mean)

110381.0 105631.0


##### Visualizing

In [15]:
label = ['Father','Mother']
value = [Father_child1_similarity_mean,Mother_child1_similarity_mean]

graph = figure(x_range=label,title="compare similarity between father and mother")

graph.vbar(x=label, top=value,width=0.5,fill_color =['red','blue'])

show(graph)


#### Decision making
- As chart shows we find similarity between children with father and mother is almost equal.